# <center>온라인 뉴스 분석</center>

### <div style="text-align: right"> 2019212840 황민섭 </div>

# 분석 배경 및 목적

### 인터넷의 영향력이 증가하고 있는 가운데, 신문-방송과 같은 전통적인 뉴스 포털보다 인터넷을 통해 제공되는 뉴스의 영향력이 커지고 있다. 
### 앞으로 더욱 더 많은신문사 닷컴의 온라인 뉴스가 증가 할 것이기에 어느 매체가 온라인 뉴스의 장점(상호작용성과 즉시성 등)을 더 가지고 있는 가에 대해 판단하기 위해 분석하였다.

---

## 분석 자료
1. 동아일보 최신 기사 200개 (6.9일 19:00 기준)

# 분석 순서

### 1. 자료 스크래핑
### 2. 기초 분석 및 교차표 생성
### 3. 토픽 분석
### 4. 계층적 군집 분석

---

### 1-1 동아일보 최신 기사 순으로 스크래핑

In [2]:
# 라이브러리
library(rvest)
library(dplyr)
library(stringr)

In [3]:
# 동아일보 url 읽기
url = 'https://www.donga.com/news/List?p='

news_page = read_html(url)

In [4]:
page_num = NULL
urls = NULL

In [5]:
# 동아일보 pagenum 설정
# ex) p=1, p=21, p=41. p=61...

for (x in 1:50) {
  page_num[x] = ((10*(x-1))*2)+1
}
page_num

[1]   1  21  41  61  81 101 121 141 161 181 201 221 241 261 281 301 321 341 361
[20] 381 401 421 441 461 481 501 521 541 561 581 601 621 641 661 681 701 721 741
[39] 761 781 801 821 841 861 881 901 921 941 961 981

In [6]:
for (y in 1:50) {
  urls[y] = paste0(url,page_num[y])
}
urls[1:10]

[1] "https://www.donga.com/news/List?p=1"  
 [2] "https://www.donga.com/news/List?p=21" 
 [3] "https://www.donga.com/news/List?p=41" 
 [4] "https://www.donga.com/news/List?p=61" 
 [5] "https://www.donga.com/news/List?p=81" 
 [6] "https://www.donga.com/news/List?p=101"
 [7] "https://www.donga.com/news/List?p=121"
 [8] "https://www.donga.com/news/List?p=141"
 [9] "https://www.donga.com/news/List?p=161"
[10] "https://www.donga.com/news/List?p=181"

In [7]:
# 기사 본문 url
d_n_urls = NULL

for (url in urls) {
  donga = read_html(url, encoding='utf8')
  d_n_urls = c(d_n_urls, donga %>% html_nodes('#content > div > div > span.tit > a:nth-child(1)')
             %>% html_attr('href'))
  

} 

head(d_n_urls)
length(d_n_urls)

[1] "https://www.donga.com/news/Entertainment/article/all/20230617/119814445/1"
[2] "https://www.donga.com/news/Society/article/all/20230617/119814444/1"      
[3] "https://www.donga.com/news/Society/article/all/20230617/119814443/1"      
[4] "https://www.donga.com/news/Economy/article/all/20230617/119814440/1"      
[5] "https://www.donga.com/news/Society/article/all/20230617/119814421/2"      
[6] "https://www.donga.com/news/Society/article/all/20230617/119814329/2"

[1] 1000

In [9]:
d_news_headline = NULL


for (n_url in d_n_urls) {
  
  donga = read_html(n_url, encoding='utf8')
  
  
  d_news_headline= c(d_news_headline, 
                   donga 
                   %>% html_nodes('#container > div.article_title > h1')
                   %>% html_text())

}

head(d_news_headline)
length(d_news_headline)

[1] "한문철 변호사 “급발진 사고 원인 밝혀 해결하는 게 꿈이다”"               
[2] "전세버스서 김밥 먹은 관광객 26명 집단 식중독 증세…병원 이송"             
[3] "건설노조 “양회동 장례 마무리 후 2차 총파업 진행”"                       
[4] "尹공약 ‘청년도약계좌’… 어느 은행이 가장 유리할까? [김도형의 돈의 뒷면]"
[5] "“대통령 죽이겠다” 국정원에 협박전화 건 60대 긴급체포"                   
[6] "엘베 막은 자전거 치웠더니…“손대지마” 적반하장 쪽지 붙었다 [e글e글]"

[1] 1000

In [10]:
d_news_maintext = NULL


for (n_url in d_n_urls) {
  
  donga = read_html(n_url, encoding='utf8')
  
  
  d_news_maintext= c(d_news_maintext, 
                   donga 
                   %>% html_nodes(xpath='//*[@id="article_txt"]/text()[2]')
                   %>% html_text())

}

head(d_news_maintext)
length(d_news_maintext)

[1] "한문철 변호사가 자신의 목표를 밝혔다. "                                                                                                      
[2] "김밥을 먹은 승객들이 단체 식중독 의심 증세를 보여 소방, 방역 당국이 대응에 나서고 있다."                                                     
[3] "\r\n            "                                                                                                                            
[4] "돈의 뒷면, 오늘은 15일부터 가입 신청을 받고 있는 청년도약계좌를 한번 뜯어보겠습니다."                                                        
[5] "국가정보원에 전화해 “대통령을 살해하겠다”고 협박한 남성이 경찰에 붙잡혔다."                                                                
[6] "공동주택 엘리베이터 앞에 세워져 있는 자전거를 치웠더니 자전거 주인으로부터 ‘파손되면 변상 조치하겠다’는 경고문을 받았다는 사연이 전해졌다."

[1] 1000

In [11]:
d_news_topic = NULL


for (n_url in d_n_urls) {
  
  donga = read_html(n_url, encoding='utf8')
  
  
  d_news_topic= c(d_news_topic, 
                   donga 
                   %>% html_nodes('#container > div.article_title > div.location > a')
                   %>% html_text())
  
#container > div.article_title > div.location
}

head(d_news_topic)
length(d_news_topic)

[1] "연예" "사회" "사회" "경제" "사회" "사회"

[1] 998

### 1-2. 데이터 전처리

In [36]:
d_news_headline = gsub(' PDF지면보기','',d_news_headline)
d_news_maintext = str_replace_all(d_news_maintext,'\r\n            ','/') 

In [37]:
head(d_news_maintext)

[1] "한문철 변호사가 자신의 목표를 밝혔다. "                                                                                                      
[2] "김밥을 먹은 승객들이 단체 식중독 의심 증세를 보여 소방, 방역 당국이 대응에 나서고 있다."                                                     
[3] "/"                                                                                                                                           
[4] "돈의 뒷면, 오늘은 15일부터 가입 신청을 받고 있는 청년도약계좌를 한번 뜯어보겠습니다."                                                        
[5] "국가정보원에 전화해 “대통령을 살해하겠다”고 협박한 남성이 경찰에 붙잡혔다."                                                                
[6] "공동주택 엘리베이터 앞에 세워져 있는 자전거를 치웠더니 자전거 주인으로부터 ‘파손되면 변상 조치하겠다’는 경고문을 받았다는 사연이 전해졌다."

In [39]:
count_news <- 998

d_all_df <- NULL
for (x in 1:count_news) {
  d_all_df[x] = paste0(d_news_headline[x], d_news_maintext[x], collapse = ' ') 
}

head(d_all_df)
length(d_all_df)

[1] "한문철 변호사 “급발진 사고 원인 밝혀 해결하는 게 꿈이다”한문철 변호사가 자신의 목표를 밝혔다. "                                                                                                                 
[2] "전세버스서 김밥 먹은 관광객 26명 집단 식중독 증세…병원 이송김밥을 먹은 승객들이 단체 식중독 의심 증세를 보여 소방, 방역 당국이 대응에 나서고 있다."                                                              
[3] "건설노조 “양회동 장례 마무리 후 2차 총파업 진행”/"                                                                                                                                                              
[4] "尹공약 ‘청년도약계좌’… 어느 은행이 가장 유리할까? [김도형의 돈의 뒷면]돈의 뒷면, 오늘은 15일부터 가입 신청을 받고 있는 청년도약계좌를 한번 뜯어보겠습니다."                                                    
[5] "“대통령 죽이겠다” 국정원에 협박전화 건 60대 긴급체포국가정보원에 전화해 “대통령을 살해하겠다”고 협박한 남성이 경찰에 붙잡혔다."                                                                               
[6] "엘베 막은 자전거 치웠더니…“손대지마” 적반하장 쪽지 붙었다 [e글e글]공동주택 엘리베이터 앞에 세워져 있는 자전거를 치웠더니 자전거 주인으로부터 ‘파손되면 변상 조치하겠다’는 경고문을 받았다는 사연이 전해졌다."

[1] 998

In [40]:
d_tit_df = as.data.frame(d_news_headline)
d_text_df = as.data.frame(d_news_maintext)
d_topic_df = as.data.frame(d_news_topic)

d_df = cbind(d_tit_df, d_text_df, d_all_df, d_topic_df)

colnames(d_df) <- c('title','text','all','topic')

head(d_df)

title,text,all,topic
한문철 변호사 “급발진 사고 원인 밝혀 해결하는 게 꿈이다”,한문철 변호사가 자신의 목표를 밝혔다.,한문철 변호사 “급발진 사고 원인 밝혀 해결하는 게 꿈이다”한문철 변호사가 자신의 목표를 밝혔다.,연예
전세버스서 김밥 먹은 관광객 26명 집단 식중독 증세…병원 이송,"김밥을 먹은 승객들이 단체 식중독 의심 증세를 보여 소방, 방역 당국이 대응에 나서고 있다.","전세버스서 김밥 먹은 관광객 26명 집단 식중독 증세…병원 이송김밥을 먹은 승객들이 단체 식중독 의심 증세를 보여 소방, 방역 당국이 대응에 나서고 있다.",사회
건설노조 “양회동 장례 마무리 후 2차 총파업 진행”,/,건설노조 “양회동 장례 마무리 후 2차 총파업 진행”/,사회
尹공약 ‘청년도약계좌’… 어느 은행이 가장 유리할까? [김도형의 돈의 뒷면],"돈의 뒷면, 오늘은 15일부터 가입 신청을 받고 있는 청년도약계좌를 한번 뜯어보겠습니다.","尹공약 ‘청년도약계좌’… 어느 은행이 가장 유리할까? [김도형의 돈의 뒷면]돈의 뒷면, 오늘은 15일부터 가입 신청을 받고 있는 청년도약계좌를 한번 뜯어보겠습니다.",경제
“대통령 죽이겠다” 국정원에 협박전화 건 60대 긴급체포,국가정보원에 전화해 “대통령을 살해하겠다”고 협박한 남성이 경찰에 붙잡혔다.,“대통령 죽이겠다” 국정원에 협박전화 건 60대 긴급체포국가정보원에 전화해 “대통령을 살해하겠다”고 협박한 남성이 경찰에 붙잡혔다.,사회
엘베 막은 자전거 치웠더니…“손대지마” 적반하장 쪽지 붙었다 [e글e글],공동주택 엘리베이터 앞에 세워져 있는 자전거를 치웠더니 자전거 주인으로부터 ‘파손되면 변상 조치하겠다’는 경고문을 받았다는 사연이 전해졌다.,엘베 막은 자전거 치웠더니…“손대지마” 적반하장 쪽지 붙었다 [e글e글]공동주택 엘리베이터 앞에 세워져 있는 자전거를 치웠더니 자전거 주인으로부터 ‘파손되면 변상 조치하겠다’는 경고문을 받았다는 사연이 전해졌다.,사회


In [41]:
write.csv(d_df,file="C:/Users/hmins/Desktop/project_df.csv")